In [1]:
!sudo -H pip install -q transformers --upgrade

In [2]:
!sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

     |████████████████████████████████| 20.0MB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 184kB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 16.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 27.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 11.4MB/s eta 0:00:01


     |████████████████████████████████| 389kB 20.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 25.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 53.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 26.7MB/s eta 0:00:01
     |████████████████████████████████| 6.2MB 47.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 25.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 74.2MB/s eta 0:00:01[K     |███████████                     | 1.3MB 74.2MB/s eta 0:00:01
     |████████████████████████████████| 204kB 68.1MB/s eta 0:00:01
     |████████████████████████████████| 389kB 66.8MB/s eta 0:00:01


     |████████████████████████████████| 81kB 11.0MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 74.4MB/s eta 0:00:01
     |████████████████████████████████| 92kB 26.4MB/s eta 0:00:01
     |████████████████████████████████| 471kB 80.3MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 60.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 20.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 32.6MB/s eta 0:00:01
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-cp38-none-any.whl size=16865 sha256=bd787809d126193ca776deb372bd03685d5d5c08d549ce2175a3166892c555c6
  Stored in directory: /root/.cache/pip/wheels/2e/10/02/9725042cf8a987b45e38bcfb106f4b0cdcc9d8968e248b93e1
  Created wheel for rouge-score: filename=rouge_score-0.1.2-cp38-none-any.whl size=24937 sha256=6aaf5daa9c69d6a2b7ad1a4cbdfcdcd8c4ab4157fd7471a4f4fae3c118dcb33a
  Stored in directory: /root/.cache/pip/wheels/df/aa/59/74f33db3bbedf322bcaadca4a43750ea5eb523bbe742d78b25
S

  Found existing installation: platformdirs 3.6.0
    Uninstalling platformdirs-3.6.0:
      Successfully uninstalled platformdirs-3.6.0
  Found existing installation: pluggy 1.0.0
    Uninstalling pluggy-1.0.0:
      Successfully uninstalled pluggy-1.0.0


In [1]:
import pandas

In [2]:
import numpy

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [4]:
!pip list | grep cuda

nvidia-cuda-cupti-cu11   11.7.101   
nvidia-cuda-nvrtc-cu11   11.7.99    
nvidia-cuda-runtime-cu11 11.7.99    


In [5]:
import pandas

In [6]:
import torch
torch.__version__

'2.0.1+cu117'

In [7]:
#model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [8]:
#quant_path = '/data/mistral/query-to-mql/exp-10/nov-20/quant-model'

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [5]:
model_path='lmsys/vicuna-7b-v1.5'

In [17]:
!pip install transformers==4.35.2

     |████████████████████████████████| 7.9MB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 25.4MB/s eta 0:00:01████████████████████████        | 2.9MB 25.4MB/s eta 0:00:01
     |████████████████████████████████| 778kB 73.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 30.8MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 68.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.8MB/s  eta 0:00:01
     |████████████████████████████████| 337kB 67.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 24.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/c8/6b/6600ac24725c7388255b2f5add93f91e58a5d7efaf4af244fdbcc11a541b/PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     |████████████████████████████████| 17.3MB 51.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 38.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 82.9MB/s eta 0:00:01
     |████████

In [1]:
!pip freeze | grep transformer

transformers==4.35.2


In [2]:
import transformers
transformers.__version__

'4.35.2'

In [6]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [7]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/pip_packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/tmp/pip_packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [9]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


ValueError: Invalid pattern: '**' can only be an entire path component

In [15]:
#model.save_quantized(quant_path)
#tokenizer.save_pretrained(quant_path)

('/data/quantization-trials/quant-model/tokenizer_config.json',
 '/data/quantization-trials/quant-model/special_tokens_map.json',
 '/data/quantization-trials/quant-model/vocab.json',
 '/data/quantization-trials/quant-model/merges.txt',
 '/data/quantization-trials/quant-model/added_tokens.json',
 '/data/quantization-trials/quant-model/tokenizer.json')

### Inference on quantized model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [14]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [15]:
import time

In [23]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 21.147445678710938


In [24]:
print(output1)

</s>what is art?

Art is a way of expressing one's self. It is a way of expressing one's thoughts, feelings, and ideas. It is a way of expressing one's self through the use of various media.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art for

### Inference on un-quantized model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
model_path = '/data/quantization-trials/merged-model'

In [20]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

time taken is : 23.784337282180786


In [26]:
print(output2)

</s>what is art?

Art is a creative expression of the human mind. It is a way of expressing the human mind through the use of various media.

The word art is derived from the Latin word artus, which means "to make". The word art is used in many different contexts, including:

- to describe the beauty of a work of art

- to describe the meaning of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work 

In [26]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [27]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out